In [ ]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split # 학습, 테스트set 구분
from sklearn.tree import export_graphviz # tree 시각화를 위해
import graphviz # tree 시각화
from sklearn.metrics import f1_score# 성능지표를 계산하기 위해 import
from sklearn.model_selection import cross_val_score, cross_validate # 교차검증

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/LGaimers/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/LGaimers/test.csv')

## 전처리

In [ ]:
col_list = train_df.columns
nan_list = []
nan_cnt = []
nan_col = []
full_list = []
for col in col_list:
    if train_df[col].isnull().sum() == 0 :
        full_list.append(col)
        continue
    nan_list.append([col, train_df[col].isnull().sum()])
    nan_cnt.append(train_df[col].isnull().sum())
    nan_col.append(col)

In [ ]:
'''모든값이 결측값이면 제거'''

del_col = []
for col in nan_list :
    if col[1] == 598 :
        del_col.append(col[0])
train_df = train_df.drop(columns=del_col)
test_df = test_df.drop(columns=del_col)
train_df.head(3)

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,NaN


In [ ]:
trainA_31 = train_df[train_df['PRODUCT_CODE']=='A_31']
train_T_O = train_df[train_df['PRODUCT_CODE']!='A_31']

testA_31 = test_df[test_df['PRODUCT_CODE']=='A_31']
test_T_O = test_df[test_df['PRODUCT_CODE']!='A_31']

In [ ]:
#trainA_31 

col_list = train_df.columns
nan_listA_31 = []
nan_cntA_31 = []
nan_colA_31 = []
full_listA_31 = []
for col in col_list:
    if trainA_31[col].isnull().sum() == 0 :
        full_listA_31.append(col)
        continue
    nan_listA_31.append([col, trainA_31[col].isnull().sum()])
    nan_cntA_31.append(trainA_31[col].isnull().sum())
    nan_colA_31.append(col)

'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listA_31 :
    if col[1] == len(trainA_31) :
        del_col.append(col[0])
trainA_31 = trainA_31.drop(columns=del_col)
testA_31 = testA_31.drop(columns=del_col)
trainA_31.head(3)

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_128,X_129,X_130,X_131,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,TRAIN_000,1,53.343333,2022-06-13 5:14,T050304,A_31,7813.0,7813.0,NaN,NaN,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN
1,TRAIN_001,2,54.181905,2022-06-13 5:22,T050307,A_31,NaN,NaN,19854.0,19854.0,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,NaN
2,TRAIN_002,1,53.126667,2022-06-13 5:30,T050304,A_31,7815.0,7815.0,NaN,NaN,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,NaN


In [ ]:
#train_T_O

col_list = train_df.columns
nan_listT_O = []
nan_cntT_O = []
nan_colT_O = []
full_listT_O = []
for col in col_list:
    if train_T_O[col].isnull().sum() == 0 :
        full_listT_O.append(col)
        continue
    nan_listT_O.append([col, train_T_O[col].isnull().sum()])
    nan_cntT_O.append(train_T_O[col].isnull().sum())
    nan_colT_O.append(col)

'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listT_O :
    if col[1] == len(train_T_O) :
        del_col.append(col[0])
train_T_O = train_T_O.drop(columns=del_col)
test_T_O = test_T_O.drop(columns=del_col)
train_T_O.head(3)

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_924,X_925,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933
22,TRAIN_022,0,51.771905,2022-06-14 8:53,T100304,T_31,2.0,102.0,0.0,45.0,...,172.5,162.8,157.235484,150.9,326.0,309.290909,290.0,13.7,13.443333,13.2
23,TRAIN_023,0,51.909048,2022-06-14 9:01,T100304,T_31,2.0,102.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,TRAIN_025,1,52.936191,2022-06-19 9:11,T100304,T_31,2.0,97.0,0.0,45.0,...,177.8,157.1,154.074194,150.9,327.0,310.854546,298.0,13.7,13.454839,13.2


In [ ]:
# train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class','Y_Quality'])
# train_quality = train_df['Y_Quality']
# train_class = train_df['Y_Class']

# test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

trainA_31_x = trainA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
testA_31_x = testA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])

train_T_O_x = train_T_O.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','PRODUCT_CODE'])
test_T_O_x = test_T_O.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])

# classification
trainA_31_y_c = trainA_31['Y_Class']
train_T_O_y_c = train_T_O['Y_Class']

# regression
trainA_31_y_r = trainA_31['Y_Quality']
train_T_O_y_r = train_T_O['Y_Quality']

In [ ]:
## 결측치 대체
trainA_31_x=trainA_31_x.fillna(-1)
testA_31_x=testA_31_x.fillna(-1)
train_T_O_x=train_T_O_x.fillna(-1)
test_T_O_x=test_T_O_x.fillna(-1)

In [ ]:
trainA_31_x

,LINE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,T050304,7813.0,7813.0,-1.0,-1.0,0.19,0.20,0.19,0.2,228.0,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,-1.0
1,T050307,-1.0,-1.0,19854.0,19854.0,0.20,0.21,0.20,0.2,413.0,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,-1.0
2,T050304,7815.0,7815.0,-1.0,-1.0,0.19,0.20,0.19,0.2,228.0,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,-1.0
3,T050307,-1.0,-1.0,19856.0,19856.0,0.20,0.21,0.20,0.2,414.0,...,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78,-1.0
4,T050304,7817.0,7817.0,-1.0,-1.0,0.19,0.20,0.18,0.2,228.0,...,182.6,383.0,367.351852,352.0,38.70,41.89,46.93,33.09,76.97,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,T050304,11920.0,11920.0,-1.0,-1.0,0.19,0.20,0.19,0.2,230.0,...,181.6,394.0,371.943396,353.0,51.71,59.64,54.61,57.05,63.18,1.0
584,T010305,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.0,-1.0,...,184.9,466.0,448.634615,432.0,-1.00,-1.00,-1.00,-1.00,-1.00,-1.0
585,T010306,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.0,-1.0,...,176.7,472.0,450.339623,432.0,-1.00,-1.00,-1.00,-1.00,-1.00,-1.0
594,T050304,14810.0,14810.0,-1.0,-1.0,0.19,0.20,0.19,0.2,304.0,...,168.7,384.0,369.811321,353.0,49.47,53.07,50.89,55.10,66.49,1.0


In [ ]:
# tmp = trainA_31_x.drop('LINE', axis=1)

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# tmp[ : ] = scaler.fit_transform(tmp[ : ])
# tmp.insert(0,'LINE',trainA_31_x['LINE'])
# tmp

,LINE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,T050304,0.293793,0.293793,0.000000,0.000000,0.991667,0.991736,0.991667,1.0,0.261714,...,0.918723,0.10,0.113966,0.119565,0.658720,0.601349,0.546491,0.575907,0.975480,0.0
1,T050307,0.000000,0.000000,0.759099,0.759099,1.000000,1.000000,1.000000,1.0,0.473143,...,0.869376,0.10,0.107939,0.119565,0.651372,0.629055,0.731477,0.596422,0.910836,0.0
2,T050304,0.293868,0.293868,0.000000,0.000000,0.991667,0.991736,0.991667,1.0,0.261714,...,0.577649,0.10,0.103476,0.119565,0.656270,0.540482,0.707865,0.615953,0.982663,0.0
3,T050307,0.000000,0.000000,0.759176,0.759176,1.000000,1.000000,1.000000,1.0,0.474286,...,0.582003,0.11,0.123562,0.119565,0.632593,0.576658,0.865820,0.518300,0.901300,0.0
4,T050304,0.293943,0.293943,0.000000,0.000000,0.991667,0.991736,0.983333,1.0,0.261714,...,0.825835,0.10,0.103810,0.108696,0.648269,0.615705,0.780492,0.559495,0.965573,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,T050304,0.448208,0.448208,0.000000,0.000000,0.991667,0.991736,0.991667,1.0,0.264000,...,0.811321,0.21,0.153185,0.119565,0.860712,0.870514,0.905553,0.952733,0.794799,1.0
584,T010305,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.859216,0.93,0.977873,0.978261,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
585,T010306,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.740203,0.99,0.996207,0.978261,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
594,T050304,0.556867,0.556867,0.000000,0.000000,0.991667,0.991736,0.991667,1.0,0.348571,...,0.624093,0.11,0.130258,0.119565,0.824135,0.776199,0.844976,0.920729,0.835789,1.0


In [ ]:
# trainA_31_x = tmp.copy()
# trainA_31_x

,LINE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,T050304,0.293793,0.293793,0.000000,0.000000,0.991667,0.991736,0.991667,1.0,0.261714,...,0.918723,0.10,0.113966,0.119565,0.658720,0.601349,0.546491,0.575907,0.975480,0.0
1,T050307,0.000000,0.000000,0.759099,0.759099,1.000000,1.000000,1.000000,1.0,0.473143,...,0.869376,0.10,0.107939,0.119565,0.651372,0.629055,0.731477,0.596422,0.910836,0.0
2,T050304,0.293868,0.293868,0.000000,0.000000,0.991667,0.991736,0.991667,1.0,0.261714,...,0.577649,0.10,0.103476,0.119565,0.656270,0.540482,0.707865,0.615953,0.982663,0.0
3,T050307,0.000000,0.000000,0.759176,0.759176,1.000000,1.000000,1.000000,1.0,0.474286,...,0.582003,0.11,0.123562,0.119565,0.632593,0.576658,0.865820,0.518300,0.901300,0.0
4,T050304,0.293943,0.293943,0.000000,0.000000,0.991667,0.991736,0.983333,1.0,0.261714,...,0.825835,0.10,0.103810,0.108696,0.648269,0.615705,0.780492,0.559495,0.965573,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,T050304,0.448208,0.448208,0.000000,0.000000,0.991667,0.991736,0.991667,1.0,0.264000,...,0.811321,0.21,0.153185,0.119565,0.860712,0.870514,0.905553,0.952733,0.794799,1.0
584,T010305,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.859216,0.93,0.977873,0.978261,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
585,T010306,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.740203,0.99,0.996207,0.978261,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
594,T050304,0.556867,0.556867,0.000000,0.000000,0.991667,0.991736,0.991667,1.0,0.348571,...,0.624093,0.11,0.130258,0.119565,0.824135,0.776199,0.844976,0.920729,0.835789,1.0


In [ ]:
## label encodeing -> 변수 선택 이후 진행

# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(trainA_31_x[i])
    trainA_31_x[i] = le.transform(trainA_31_x[i])
    
    for label in np.unique(testA_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    testA_31_x[i] = le.transform(testA_31_x[i]) 
print('trainA_31_x : Done.')

# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_T_O_x[i])
    train_T_O_x[i] = le.transform(train_T_O_x[i])
    
    for label in np.unique(test_T_O_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_T_O_x[i] = le.transform(test_T_O_x[i]) 
print('train_T_O_x : Done.')

trainA_31_x : Done.
train_T_O_x : Done.


## Linear Regression : feature 생성

In [ ]:
import statsmodels.api as sm

#define response variable
y = trainA_31_y_r.copy()

#define predictor variables
x = trainA_31_x.copy()

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()

#view model summary
print(model.summary())

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/regression/linear_model.py:1728: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/lib/python3.8/dist-packages/statsmodels/regression/linear_model.py:1728: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/lib/python3.8/dist-packages/statsmodels/regression/linear_model.py:1650: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:1452: RuntimeWarning: invalid value encountered in multiply
  cov_p = self.normalized_cov_params

                            OLS Regression Results                            
Dep. Variable:              Y_Quality   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 05 Feb 2023   Prob (F-statistic):                nan
Time:                        08:52:12   Log-Likelihood:                 6706.9
No. Observations:                 249   AIC:                        -1.292e+04
Df Residuals:                       0   BIC:                        -1.204e+04
Df Model:                         248                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4713        inf          0        n

In [ ]:
# 학습, 테스트 데이터 분리 (0.7:0.3)
seed = 2022
train_x_r, valid_x_r, train_y_r, valid_y_r = train_test_split(trainA_31_x, trainA_31_y_r, test_size=0.3, random_state=seed)
train_x_c, valid_x_c, train_y_c, valid_y_c = train_test_split(trainA_31_x, trainA_31_y_c, test_size=0.3, random_state=seed)

In [ ]:
from sklearn.linear_model import LinearRegression

LR = LinearRegression()
LR.fit(train_x_r, train_y_r)

LinearRegression()

In [ ]:
LR_pred = LR.predict(valid_x_r)
LR_pred

array([53.92428828, 53.38972841, 54.0775801 , 52.54714639, 52.82166653,
       51.96773122, 54.63437637, 52.52247573, 53.27967836, 54.88365435,
       51.42511362, 53.52968168, 54.17642451, 51.8095269 , 53.91735643,
       51.49242085, 51.73710009, 57.80144434, 55.44056608, 53.72274489,
       52.48151079, 53.80909497, 51.81429821, 52.48882216, 54.68447847,
       54.17584587, 52.72701884, 54.01896974, 53.5059253 , 52.77011032,
       51.67447301, 53.15555863, 54.23306795, 54.33283633, 52.43400907,
       54.06299908, 52.95356781, 54.4473898 , 53.49052988, 52.96828991,
       56.16006816, 52.96592739, 50.02274918, 53.21088199, 53.07117815,
       51.6438069 , 52.97395267, 54.67822586, 52.54859672, 54.89214337,
       52.18489808, 53.80173739, 52.4253291 , 52.97015396, 52.01667021,
       55.14931539, 53.97436868, 53.39489033, 50.0891771 , 52.16582975,
       49.30854421, 51.8184376 , 52.97365516, 53.73860782, 53.47846324,
       52.88922409, 52.55166352, 53.94347512, 52.64153376, 52.20

In [ ]:
LR_class = []

for i in LR_pred:
  if i<=0.52507*distance: LR_class.append(0)
  elif 0.52507*distance<i<0.5349*distance: LR_class.append(1)
  else: LR_class.append(2)

np.array(LR_class)

array([2, 1, 2, 1, 1, 0, 2, 1, 1, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2,
       0, 0, 2, 2, 1, 2, 2, 1, 0, 1, 2, 2, 0, 2, 1, 2, 2, 1, 2, 1, 0, 1,
       1, 0, 1, 2, 1, 2, 0, 2, 0, 1, 0, 2, 2, 1, 0, 0, 0, 0, 1, 2, 1, 1,
       1, 2, 1, 0, 2, 2, 0, 2, 0])

In [ ]:
LR_f1 = f1_score(valid_y_c, LR_class, average='micro')
LR_f1

0.3333333333333333

## 다중공선성 제거

In [ ]:
vif_trainA_31_x_r = pd.read_csv('/content/drive/MyDrive/LGaimers/input/vif_trainA_31_x.csv')
vif_trainA_31_x_r

,VIF Factor,features
0,inf,LINE
1,inf,X_2108
2,inf,X_2107
3,inf,X_2106
4,inf,X_2105
...,...,...
2112,NaN,X_1823
2113,NaN,X_2052
2114,NaN,X_2053
2115,NaN,X_2054


In [ ]:
vif_trainA_31_x_r['VIF Factor'].unique()

array([inf, nan])

In [ ]:
problem = vif_trainA_31_x_r[vif_trainA_31_x_r['VIF Factor'] > 9]
problem

,VIF Factor,features
0,inf,LINE
1,inf,X_2108
2,inf,X_2107
3,inf,X_2106
4,inf,X_2105
...,...,...
2085,inf,X_1384
2086,inf,X_1383
2087,inf,X_1382
2088,inf,X_1380


In [ ]:
a = trainA_31_x.copy()
a = a.T.drop_duplicates()
a = a.T
a

,LINE,X_128,X_129,X_130,X_132,X_133,X_134,X_135,X_136,X_137,...,X_2799,X_2800,X_2801,X_2837,X_2839,X_2840,X_2841,X_2842,X_2843,X_2871
0,2.0,0.293793,0.293793,0.000000,0.991667,0.991736,0.991667,1.0,0.261714,0.261714,...,0.896552,0.884780,0.84,0.871528,0.923313,0.993789,0.963171,0.995870,0.999990,0.0
1,3.0,0.000000,0.000000,0.759099,1.000000,1.000000,1.000000,1.0,0.473143,0.474286,...,0.931034,0.963328,1.00,0.781250,0.920245,1.000000,0.880601,0.996685,0.999966,0.0
2,2.0,0.293868,0.293868,0.000000,0.991667,0.991736,0.991667,1.0,0.261714,0.261714,...,0.931034,0.923130,0.92,0.871528,0.923313,0.934161,0.962937,0.995871,0.999990,0.0
3,3.0,0.000000,0.000000,0.759176,1.000000,1.000000,1.000000,1.0,0.474286,0.474286,...,0.931034,0.962849,0.96,0.736979,0.921779,0.997516,0.962468,0.996745,0.999966,0.0
4,2.0,0.293943,0.293943,0.000000,0.991667,0.991736,0.983333,1.0,0.261714,0.261714,...,0.931034,0.924499,0.92,0.871528,0.924847,0.934161,0.963406,0.995872,0.999998,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2.0,0.448208,0.448208,0.000000,0.991667,0.991736,0.991667,1.0,0.264000,0.264000,...,0.965517,0.934087,0.92,0.868056,0.918712,0.993789,0.963641,0.995722,0.999966,1.0
584,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
585,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
594,2.0,0.556867,0.556867,0.000000,0.991667,0.991736,0.991667,1.0,0.348571,0.348571,...,0.965517,0.941669,0.92,0.825521,0.918712,0.993789,0.975604,0.995663,0.999966,1.0


In [ ]:
import statsmodels.api as sm

#define response variable
y = trainA_31_y_r.copy()

#define predictor variables
x = a.copy()

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()

#view model summary
print(model.summary())

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/statsmodels/regression/linear_model.py:1728: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/lib/python3.8/dist-packages/statsmodels/regression/linear_model.py:1728: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/lib/python3.8/dist-packages/statsmodels/regression/linear_model.py:1650: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


                            OLS Regression Results                            
Dep. Variable:              Y_Quality   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 05 Feb 2023   Prob (F-statistic):                nan
Time:                        08:52:28   Log-Likelihood:                 6822.8
No. Observations:                 249   AIC:                        -1.315e+04
Df Residuals:                       0   BIC:                        -1.227e+04
Df Model:                         248                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7278        inf          0        n

In [ ]:
# 학습, 테스트 데이터 분리 (0.7:0.3)
seed = 2022
train_x_r, valid_x_r, train_y_r, valid_y_r = train_test_split(a, trainA_31_y_r, test_size=0.3, random_state=seed)
train_x_c, valid_x_c, train_y_c, valid_y_c = train_test_split(a, trainA_31_y_c, test_size=0.3, random_state=seed)

In [ ]:
from sklearn.linear_model import LinearRegression

LR = LinearRegression()
LR.fit(train_x_r, train_y_r)

LinearRegression()

In [ ]:
LR_pred = LR.predict(valid_x_r)
LR_pred

array([53.95298405, 53.40502008, 53.88308656, 52.82612283, 52.88945918,
       51.98714713, 54.86015798, 52.37941677, 53.29293663, 54.81276654,
       51.5037625 , 53.56394682, 54.19196782, 51.81611202, 54.01471542,
       51.33004931, 51.77688672, 58.26967892, 55.54929234, 53.82355054,
       52.44729805, 53.7662445 , 51.90954461, 52.38701409, 54.69827304,
       54.21000975, 52.40849305, 54.15103407, 53.55740012, 52.88468383,
       51.84098262, 52.88167991, 54.14186595, 54.4940942 , 52.45370344,
       54.0253864 , 52.92191307, 54.40171307, 53.4073443 , 52.81606122,
       56.0682721 , 52.88709104, 49.87814569, 52.99383381, 53.21171319,
       51.63511969, 52.77979369, 54.54731536, 52.39915966, 55.07433966,
       52.1112634 , 53.85372569, 52.75061788, 52.78104859, 51.77149971,
       55.47021689, 54.07990549, 53.61973701, 50.19775146, 52.31903959,
       49.51705679, 51.81276603, 52.91518205, 53.9908212 , 53.40768857,
       52.8613763 , 52.50224967, 54.02913261, 52.61043929, 52.24

In [ ]:
LR_class = []

for i in LR_pred:
  if i<=0.52507*distance: LR_class.append(0)
  elif 0.52507*distance<i<0.5349*distance: LR_class.append(1)
  else: LR_class.append(2)

np.array(LR_class)

array([2, 1, 2, 1, 1, 0, 2, 0, 1, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2,
       0, 0, 2, 2, 0, 2, 2, 1, 0, 1, 2, 2, 0, 2, 1, 2, 1, 1, 2, 1, 0, 1,
       1, 0, 1, 2, 0, 2, 0, 2, 1, 1, 0, 2, 2, 2, 0, 0, 0, 0, 1, 2, 1, 1,
       0, 2, 1, 0, 2, 2, 0, 2, 0])

In [ ]:
LR_f1 = f1_score(valid_y_c, LR_class, average='micro')
LR_f1

0.36

In [ ]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(a.values, i) for i in range(a.shape[1])]
# vif["features"] = a.columns
# vif = vif.sort_values("VIF Factor").reset_index(drop=True)
# vif

In [ ]:
# vif[vif['VIF Factor'] < 9]

In [ ]:
## 후진 소거법
variables = a.columns.tolist() ## 설명 변수 리스트
 
y = trainA_31_y_r ## 반응 변수
selected_variables = variables ## 초기에는 모든 변수가 선택된 상태
sl_remove = 0.05
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(selected_variables) > 0:
    X = sm.add_constant(a[selected_variables])
    p_vals = sm.OLS(y,X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
    max_pval = p_vals.max() ## 최대 p-value
    if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
        remove_variable = p_vals.idxmax()
        selected_variables.remove(remove_variable)
 
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(a[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
len(selected_variables)

1187

In [ ]:
new = a.copy()
new = new[selected_variables]
new

,LINE,X_128,X_129,X_130,X_132,X_133,X_134,X_135,X_136,X_137,...,X_2799,X_2800,X_2801,X_2837,X_2839,X_2840,X_2841,X_2842,X_2843,X_2871
0,2.0,0.293793,0.293793,0.000000,0.991667,0.991736,0.991667,1.0,0.261714,0.261714,...,0.896552,0.884780,0.84,0.871528,0.923313,0.993789,0.963171,0.995870,0.999990,0.0
1,3.0,0.000000,0.000000,0.759099,1.000000,1.000000,1.000000,1.0,0.473143,0.474286,...,0.931034,0.963328,1.00,0.781250,0.920245,1.000000,0.880601,0.996685,0.999966,0.0
2,2.0,0.293868,0.293868,0.000000,0.991667,0.991736,0.991667,1.0,0.261714,0.261714,...,0.931034,0.923130,0.92,0.871528,0.923313,0.934161,0.962937,0.995871,0.999990,0.0
3,3.0,0.000000,0.000000,0.759176,1.000000,1.000000,1.000000,1.0,0.474286,0.474286,...,0.931034,0.962849,0.96,0.736979,0.921779,0.997516,0.962468,0.996745,0.999966,0.0
4,2.0,0.293943,0.293943,0.000000,0.991667,0.991736,0.983333,1.0,0.261714,0.261714,...,0.931034,0.924499,0.92,0.871528,0.924847,0.934161,0.963406,0.995872,0.999998,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2.0,0.448208,0.448208,0.000000,0.991667,0.991736,0.991667,1.0,0.264000,0.264000,...,0.965517,0.934087,0.92,0.868056,0.918712,0.993789,0.963641,0.995722,0.999966,1.0
584,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
585,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
594,2.0,0.556867,0.556867,0.000000,0.991667,0.991736,0.991667,1.0,0.348571,0.348571,...,0.965517,0.941669,0.92,0.825521,0.918712,0.993789,0.975604,0.995663,0.999966,1.0


In [ ]:
# 학습, 테스트 데이터 분리 (0.7:0.3)
seed = 2022
train_x_r, valid_x_r, train_y_r, valid_y_r = train_test_split(new, trainA_31_y_r, test_size=0.3, random_state=seed)
train_x_c, valid_x_c, train_y_c, valid_y_c = train_test_split(new, trainA_31_y_c, test_size=0.3, random_state=seed)

In [ ]:
from sklearn.linear_model import LinearRegression

LR = LinearRegression()
LR.fit(train_x_r, train_y_r)

LinearRegression()

In [ ]:
LR_pred = LR.predict(valid_x_r)
LR_pred

array([53.95298405, 53.40502008, 53.88308656, 52.82612283, 52.88945918,
       51.98714713, 54.86015798, 52.37941677, 53.29293663, 54.81276654,
       51.5037625 , 53.56394682, 54.19196782, 51.81611202, 54.01471542,
       51.33004931, 51.77688672, 58.26967892, 55.54929234, 53.82355054,
       52.44729805, 53.7662445 , 51.90954461, 52.38701409, 54.69827304,
       54.21000975, 52.40849305, 54.15103407, 53.55740012, 52.88468383,
       51.84098262, 52.88167991, 54.14186595, 54.4940942 , 52.45370344,
       54.0253864 , 52.92191307, 54.40171307, 53.4073443 , 52.81606122,
       56.0682721 , 52.88709104, 49.87814569, 52.99383381, 53.21171319,
       51.63511969, 52.77979369, 54.54731536, 52.39915966, 55.07433966,
       52.1112634 , 53.85372569, 52.75061788, 52.78104859, 51.77149971,
       55.47021689, 54.07990549, 53.61973701, 50.19775146, 52.31903959,
       49.51705679, 51.81276603, 52.91518205, 53.9908212 , 53.40768857,
       52.8613763 , 52.50224967, 54.02913261, 52.61043929, 52.24

In [ ]:
LR_class = []

for i in LR_pred:
  if i<=0.52507*distance: LR_class.append(0)
  elif 0.52507*distance<i<0.5349*distance: LR_class.append(1)
  else: LR_class.append(2)

np.array(LR_class)

array([2, 1, 2, 1, 1, 0, 2, 0, 1, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2,
       0, 0, 2, 2, 0, 2, 2, 1, 0, 1, 2, 2, 0, 2, 1, 2, 1, 1, 2, 1, 0, 1,
       1, 0, 1, 2, 0, 2, 0, 2, 1, 1, 0, 2, 2, 2, 0, 0, 0, 0, 1, 2, 1, 1,
       0, 2, 1, 0, 2, 2, 0, 2, 0])

In [ ]:
LR_f1 = f1_score(valid_y_c, LR_class, average='micro')
LR_f1

0.36

In [ ]:
# 학습, 테스트 데이터 분리 (0.7:0.3)
seed = 2022
train_x_r, valid_x_r, train_y_r, valid_y_r = train_test_split(trainA_31_x, trainA_31_y_r, test_size=0.3, random_state=seed)
train_x_c, valid_x_c, train_y_c, valid_y_c = train_test_split(trainA_31_x, trainA_31_y_c, test_size=0.3, random_state=seed)

In [ ]:
LR_class = []

for i in trainA_31['Y_Quality']:
  if i<=0.52507*distance: LR_class.append(0)
  elif 0.52507*distance<i<0.5349*distance: LR_class.append(1)
  else: LR_class.append(2)

np.array(LR_class)

array([1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2,
       2, 2, 0, 2, 2, 1, 0, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1, 1, 1, 0, 2, 2, 2, 2, 1, 1, 1, 1,
       2, 2, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 0, 2, 1, 2, 2, 2, 2, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 1, 1, 1, 0, 2, 2, 1, 2, 1, 1, 1,
       2, 2, 2, 0, 1, 0, 1, 0, 2, 2, 1, 1, 0, 1, 0, 2, 2, 2, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 0, 0, 0, 0, 0, 1, 0,
       1, 2, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 2, 2, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 2, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [ ]:
LR_f1 = f1_score(trainA_31['Y_Class'], LR_class, average='micro')
LR_f1

1.0

# DecisionTree 학습

In [ ]:
DT_train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class'])
DT_train_y = train_df['Y_Class']

DT_test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [ ]:
DT_train_x = DT_train_x.fillna(0)
DT_test_x = DT_test_x.fillna(0)

In [ ]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(DT_train_x[i])
    DT_train_x[i] = le.transform(DT_train_x[i])
    
    for label in np.unique(DT_test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    DT_test_x[i] = le.transform(DT_test_x[i]) 
print('Done.')

Done.


In [ ]:
# 학습, 테스트 데이터 분리 
seed = 2022
DT_train_x, DT_valid_x, DT_train_y, DT_valid_y = DT_train_x[:400], DT_train_x[400:], DT_train_y[:400], DT_train_y[400:]

In [ ]:
DT_train_x

,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,0.533433,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.34,40.89,32.56,34.09,77.77,0.0,0.0,0.0,0.0,0.0
1,0.541819,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.89,42.82,43.92,35.34,72.55,0.0,0.0,0.0,0.0,0.0
2,0.531267,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.19,36.65,42.47,36.53,78.35,0.0,0.0,0.0,0.0,0.0
3,0.537325,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,37.74,39.17,52.17,30.58,71.78,0.0,0.0,0.0,0.0,0.0
4,0.531590,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.70,41.89,46.93,33.09,76.97,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.532863,5,2,2.0,91.0,0.0,45.0,10.0,0.0,50.0,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
396,0.526921,5,2,2.0,96.0,0.0,45.0,10.0,0.0,51.0,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
397,0.531414,4,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
398,0.528348,4,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0


In [ ]:
DT_valid_x

,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
400,0.531684,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
401,0.529641,4,2,1.0,94.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
402,0.526705,5,2,2.0,88.0,0.0,45.0,10.0,0.0,54.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
403,0.530411,5,2,2.0,93.0,0.0,45.0,10.0,0.0,53.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
404,0.500856,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,0.526546,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
594,0.524022,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49.47,53.07,50.89,55.1,66.49,1.0,0.0,0.0,0.0,0.0
595,0.521289,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,1.0,0.0,0.0,0.0,0.0
596,0.531375,4,1,40.0,94.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [ ]:
DT = DecisionTreeClassifier(random_state=seed)
DT.fit(DT_train_x, DT_train_y)

DecisionTreeClassifier(random_state=2022)

In [ ]:
valid_x.insert(0, 'Y_Quality',LR_pred)
valid_x

,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
400,0.541169,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
401,0.533416,4,2,1.0,94.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
402,0.539708,5,2,2.0,88.0,0.0,45.0,10.0,0.0,54.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
403,0.532134,5,2,2.0,93.0,0.0,45.0,10.0,0.0,53.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
404,0.495551,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,0.919561,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
594,0.267534,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49.47,53.07,50.89,55.1,66.49,1.0,0.0,0.0,0.0,0.0
595,0.248942,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,1.0,0.0,0.0,0.0,0.0
596,-0.326478,4,1,40.0,94.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [ ]:
DT_valid = DT.predict(valid_x)
DT_valid

array([2, 1, 2, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1,
       0, 1, 0, 1, 2, 0, 2, 0, 1, 1, 1, 0, 1, 0, 0, 2, 1, 0, 2, 1, 0, 1,
       1, 1, 1, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 1, 1, 2, 2, 1, 1, 1, 2,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 2, 0, 1, 0, 1, 2, 0, 2, 0, 1, 2,
       2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0,
       0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2])

In [ ]:
DT_f1 = f1_score(DT_valid_y, DT_valid, average='macro')
DT_f1

0.34581768974272653